In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
import os
import h5py
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
for (root,dirs,files) in os.walk('E:/minor_3_dataset/Clean/'):
    print()

print(files)
path=""
# Second method to  load hdf5 file in case of error
# df1 = pd.DataFrame(np.array(h5py.File(path)['air']))
X = pd.read_hdf(path)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.20, random_state=0)
del X
del y

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

In [ ]:
# Optional
# Normalizing data for numerical stability
from sklearn.preprocessing import StandardScaler

ss_train = StandardScaler()
X_train = ss_train.fit_transform(X_train)

ss_test = StandardScaler()
X_test = ss_test.fit_transform(X_test)

# Machine learning methods

In [ ]:
models = {}
history = {}

# Logistic Regression
from sklearn.linear_model import LogisticRegression
models['Logistic Regression'] = LogisticRegression(max_iter=3000)

# Support Vector Machines
from sklearn.svm import LinearSVC
models['Support Vector Machines'] = LinearSVC()

# Decision Trees
from sklearn.tree import DecisionTreeClassifier
models['Decision Trees'] = DecisionTreeClassifier()

# Random Forest
from sklearn.ensemble import RandomForestClassifier
models['Random Forest'] = RandomForestClassifier()

# Naive Bayes
from sklearn.naive_bayes import GaussianNB
models['Naive Bayes'] = GaussianNB()

# K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
models['K-Nearest Neighbor'] = KNeighborsClassifier()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

accuracy, precision, recall = {}, {}, {}

for key in models.keys():
    
    # Fit the classifier
    models[key].fit(X_train, y_train)
    
    # Make predictions
    predictions = models[key].predict(X_test)
    
    # Calculate metrics
    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test,average='micro')
    recall[key] = recall_score(predictions, y_test,average='micro')

In [ ]:
df_model = pd.DataFrame(index=models.keys(), columns=['Accuracy', 'Precision', 'Recall'])
df_model['Accuracy'] = accuracy.values()
df_model['Precision'] = precision.values()
df_model['Recall'] = recall.values()

df_model

In [ ]:
ax = df_model.plot.barh()
ax.legend(
    ncol=len(models.keys()), 
    bbox_to_anchor=(0, 1), 
    loc='lower left', 
    prop={'size': 14}
)
plt.tight_layout()

# Deep Learning based methods

In [ ]:
import time
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.layers import LSTM,Dropout,Dense,TimeDistributed,Conv1D,MaxPooling1D,InputLayer,BatchNormalization

In [ ]:
n_timesteps, n_features, n_outputs = 28800, 6000, 18 
# Rows,      Columns,    Output classes

y_train = tf.keras.utils.to_categorical(y_train, n_outputs)
y_test = tf.keras.utils.to_categorical(y_test, n_outputs)

number_epoch = 25
batch_length = 16
inp_shape=(6000,)
inner_activation_fun = 'relu'
outer_activation_fun = 'softmax'
optimizer_loss_fun = 'categorical_crossentropy'
optimizer_algorithm = 'adam'
opt = keras.optimizers.Adam(learning_rate=0.001)


## Model 1

In [ ]:
model = Sequential(
    [
        Input(shape = inp_shape),
        Flatten(),
        Dense(256,activation='relu'),
        Dropout(0.3),
        Dense(256,activation='relu'),
        Dropout(0.3),
        Dense(256,activation='relu'),
        Dropout(0.3),
        Dense(256,activation='relu'),
        Dropout(0.3),
        Dense(n_outputs,activation = "softmax")
    ]
)

model.compile(loss = "categorical_crossentropy", optimizer = "adam" , metrics = ["accuracy"])
model.summary()

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=False)

In [ ]:
start_tme = time.time()
history = model.fit(X_train, y_train, epochs=number_epoch, batch_size=batch_length, validation_data=(X_test, y_test))
end_time = time.time()
print(history.history)

In [ ]:
score = model.evaluate(X_test,y_test,verbose = 0)
print("Test loss : ", score[0])
print("Test Accuracy : ", score[1])
print("Training Time : ",end_time - start_tme)

In [ ]:
y_pred = model.predict(X_test)
ytest = np.argmax(y_test,axis=1)
pred = np.argmax(y_pred, axis=1)

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
print(classification_report(ytest,pred))
cm = confusion_matrix(ytest,pred)
plot= ConfusionMatrixDisplay(cm)
plot.plot()
plt.show()

## Model 2

In [ ]:
number_inner_layers = 5
number_inner_neurons = 2048
show_inter_results = 0

model = Sequential()
for i in range(number_inner_layers):
    number_inner_neurons=number_inner_neurons/2
    model.add(Dense(int(number_inner_neurons), input_dim=inp_shape, activation=inner_activation_fun))
    model.add(Dropout(0.3))
model.add(Dense(n_outputs, activation=outer_activation_fun))

model.summary()

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=False)

In [ ]:
model.compile(loss=optimizer_loss_fun, optimizer=optimizer_algorithm, metrics=['accuracy'])

In [ ]:
#Training
start_tme = time.time()
history = model.fit(X_train, y_train, epochs=number_epoch, batch_size=batch_length, validation_data=(X_test, y_test))
end_time = time.time()
print(history.history)

In [ ]:
#Evaluation
score = model.evaluate(X_test,y_test,verbose = 0)
print("Test loss : ", score[0])
print("Test Accuracy : ", score[1])
print("Training Time : ",end_time - start_tme)

In [ ]:
y_pred = model.predict(X_test)
ytest = np.argmax(y_test,axis=1)
pred = np.argmax(y_pred, axis=1)

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
print(classification_report(ytest,pred))
cm = confusion_matrix(ytest,pred)
plot= ConfusionMatrixDisplay(cm)
plot.plot()
# plt.grid(False)
plt.show()

In [ ]:
# del pred
# del y_pred
# del ytest

## CNN Model 1

In [ ]:

# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=inp_shape))
# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=n_outputs, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')


In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=False)

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_length, epochs=number_epoch, validation_data=(X_test, y_test))
print(history.history)

In [ ]:
score = model.evaluate(X_test,y_test,verbose = 0)
print("Test loss : ", score[0])
print("Test Accuracy : ", score[1])

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
y_pred = model.predict(X_test)
ytest = np.argmax(y_test,axis=1)
pred = np.argmax(y_pred, axis=1)

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()
cm = confusion_matrix(ytest,pred)
plot= ConfusionMatrixDisplay(cm)
plot.plot()
# plt.grid(False)
plt.show()

In [ ]:
del pred
del y_pred
del ytest

## CNN model 2

In [ ]:

X_train=X_train.reshape(28800,6000,1)
y_train=y_train.reshape(28800,18)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=inp_shape))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.3))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()


In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True,show_layer_names=False)

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_length, epochs=number_epoch, validation_data=(X_test, y_test))
print(history.history)

In [ ]:
score = model.evaluate(X_test,y_test,verbose = 0)
print("Test loss : ", score[0])
print("Test Accuracy : ", score[1])

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()
cm = confusion_matrix(ytest,pred)
plot= ConfusionMatrixDisplay(cm)
plot.plot()
# plt.grid(False)
plt.show()

## LSTM + CNN based model

In [ ]:
X_train = X_train.reshape((28800,6000, 1))
y_train = y_train.reshape((28800,18))

n_timesteps, n_features, n_outputs = 28800, 6000, 18
inp_shape = (n_features,1)
model = Sequential(name='DC')

# LFLB1
model.add(Conv1D(filters = 256,kernel_size = (3),strides=1,padding='same',data_format='channels_last',input_shape=inp_shape))	
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))
	#LFLB2
model.add(Conv1D(filters=256, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))
	#LFLB3
model.add(Conv1D(filters=256, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))
	#LFLB4
model.add(Conv1D(filters=256, kernel_size = 3, strides=1,padding='same'))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(MaxPooling1D(pool_size = 4, strides = 4))
	#LSTM
model.add(LSTM(units=128)) 
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
#FC
model.add(Dense(units=18,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

In [ ]:
#Training
start_tme = time.time()
history = model.fit(X_train, y_train, epochs=number_epoch, batch_size=batch_length, validation_data=(X_test, y_test))
end_time = time.time()
print(history.history)

In [ ]:
score = model.evaluate(X_test,y_test,verbose = 0)
print("Test loss : ", score[0])
print("Test Accuracy : ", score[1])
print("Training Time : ",end_time - start_tme)

In [ ]:
y_pred = model.predict(X_test)
ytest = np.argmax(y_test,axis=1)
pred = np.argmax(y_pred, axis=1)

In [ ]:
plt.plot(history.history['accuracy'],color='red')
plt.plot(history.history['val_accuracy'])
plt.show()

In [ ]:
print(classification_report(ytest,pred))
cm = confusion_matrix(ytest,pred)
plot= ConfusionMatrixDisplay(cm)
plot.plot()
# plt.grid(False)
plt.show()